## Аггрегация разметки датасета ruTiE

В разметке были 430 бинарных вопроса в двух сеттингах:
- с показом истории ответов разметчика на все предыдущие вопросы
- без показа истории

Контроль качества производился в ручном режиме на стороне АБК, потому этап фильтрации пропускается.

В качестве метрики используется Accuracy.

In [1]:
import pandas as pd
import json
import ast

In [2]:
pool1 = pd.read_csv('pool1.tsv', sep='\t')
pool2 = pd.read_csv('pool2.csv', sep=';')

При наличии истории разметчикам предлагалось на основании имеющихся предыдущих вопросов и своих ответов на них, а также одного нового вопроса с двумя вариантами ответа ответить на вопрос, какой ответ является правильным.

В отсутствие истории разметчикам предлагалось на основании только одного вопроса с двумя вариантами ответа ответить на вопрос, какой ответ является правильным.

Вход: 
- question (пример: `Сколько звезд на небе?`).
- answer_1 (пример: `Несчетное количество`).
- answer_2 (пример: `Звезд не существует`).

Выход:
- OUTPUT:answer_output (целое число: `1` или `2`).

In [4]:
pool1.head(1)

,INPUT:dialog,OUTPUT:answer,OUTPUT:result,ASSIGNMENT:time_spent,ASSIGNMENT:date_start,ASSIGNMENT:date_end,ASSIGNMENT:user_id,ASSIGNMENT:result_id,ASSIGNMENT:data_id,ASSIGNMENT:status,ASSIGNMENT:worker_id,ASSIGNMENT:reward,ASSIGNMENT:assignment_id,ACCEPT:verdict,ACCEPT:comment
0,"[{""answer_1"":""Четыре"",""answer_2"":""Сто"",""questi...",NaN,"[{""question"":""Сколько ножек у стола?: Четыре и...",36077,2023-11-10 09:25:22,2023-11-10 19:26:39,21222,236363631,65553487,SUBMITTED,21222,0,9246116,NaN,NaN


Формат пула необычный, так как разметка велась на платформе АБТ.

В функции предобработки переменные q1, q2, q3, q4, q5 использовались для проверки порядка ответа на вопросы, но он везде одинаковый.

In [5]:
def process_pool(pool1):
    q1 = [ast.literal_eval(pool1.iloc[0]['OUTPUT:result'])[i]['question'] for i in range(len(ast.literal_eval(pool1.iloc[0]['OUTPUT:result'])))]
    a1 = [ast.literal_eval(pool1.iloc[0]['OUTPUT:result'])[i]['answer'] for i in range(len(ast.literal_eval(pool1.iloc[0]['OUTPUT:result'])))]

    q2 = [ast.literal_eval(pool1.iloc[1]['OUTPUT:result'])[i]['question'] for i in range(len(ast.literal_eval(pool1.iloc[1]['OUTPUT:result'])))]
    a2 = [ast.literal_eval(pool1.iloc[1]['OUTPUT:result'])[i]['answer'] for i in range(len(ast.literal_eval(pool1.iloc[1]['OUTPUT:result'])))]

    q3 = [ast.literal_eval(pool1.iloc[2]['OUTPUT:result'])[i]['question'] for i in range(len(ast.literal_eval(pool1.iloc[2]['OUTPUT:result'])))]
    a3 = [ast.literal_eval(pool1.iloc[2]['OUTPUT:result'])[i]['answer'] for i in range(len(ast.literal_eval(pool1.iloc[2]['OUTPUT:result'])))]

    q4 = [ast.literal_eval(pool1.iloc[3]['OUTPUT:result'])[i]['question'] for i in range(len(ast.literal_eval(pool1.iloc[3]['OUTPUT:result'])))]
    a4 = [ast.literal_eval(pool1.iloc[3]['OUTPUT:result'])[i]['answer'] for i in range(len(ast.literal_eval(pool1.iloc[3]['OUTPUT:result'])))]

    q5 = [ast.literal_eval(pool1.iloc[4]['OUTPUT:result'])[i]['question'] for i in range(len(ast.literal_eval(pool1.iloc[4]['OUTPUT:result'])))]
    a5 = [ast.literal_eval(pool1.iloc[4]['OUTPUT:result'])[i]['answer'] for i in range(len(ast.literal_eval(pool1.iloc[4]['OUTPUT:result'])))]

    assert q1 == q2 == q3 == q4 == q5

    p1 = pd.DataFrame({'question': q1, 'a1': a1, 'q2': q2, 'a2': a2, 'q3': q3, 'a3': a3, 'q4': q4, 'a4': a4, 'q5': q5, 'a5': a5})
    mapping = {'answer_1': 1, 'answer_2': 2}
    ans1 = p1[['question', 'a1', 'a2', 'a3', 'a4', 'a5']].copy()
    ans1['cons'] = ans1[['a1', 'a2', 'a3', 'a4', 'a5']].apply(lambda x: x.value_counts()[0], axis=1)
    ans1['a1'] = ans1['a1'].map(mapping)
    ans1['a2'] = ans1['a2'].map(mapping)
    ans1['a3'] = ans1['a3'].map(mapping)
    ans1['a4'] = ans1['a4'].map(mapping)
    ans1['a5'] = ans1['a5'].map(mapping)
    ans1['pred'] = ans1[['a1', 'a2', 'a3', 'a4', 'a5']].apply(lambda x: x.value_counts().index[0], axis=1)

    return ans1

In [6]:
df1 = process_pool(pool1)
df2 = process_pool(pool2)

Собираем все ответы в одну таблицу.

In [8]:
df = pd.DataFrame({
    'question': df1['question'],
    'history_cons': df1['cons'],
    'history_lb': df1['pred'],
    'no_history_cons': df2['cons'],
    'no_history_lb': df2['pred'],
})
df.head(5)

,question,history_cons,history_lb,no_history_cons,no_history_lb
0,Сколько ножек у стола?: Четыре или Сто?,5,1,5,1
1,А у осьминога?: Четыре или Восемь?,5,2,5,2
2,А у червяка сколько?: У червяка ног не бывает ...,5,1,5,1
3,Какого цвета море?: Обычно малиновое или Чаще ...,5,2,5,2
4,А листва?: Зелёная или Чёрная?,5,1,5,1


В колонках `history_cons` и `no_history_cons` числа для определения согласованности разметки - большинство по какой-либо категории (1 или 2).

In [9]:
pd.DataFrame({
    "С показом истории": df['history_cons'].value_counts(),
    "Без показа истории": df['no_history_cons'].value_counts(),
})

,С показом истории,Без показа истории
5,294,359
4,93,51
3,43,20


Согласованность и не могла быть меньше 3 человек на задание (ведь перекрытие всегда 5, а меток всего 2).

Теперь соединяем с правильными метками для подсчета метрик.

In [10]:
with open('task.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [12]:
test = data['data']['test'][0]
true = [test[i]['outputs'] for i in range(len(test))]
df['true'] = pd.Series(true).astype(int)
df.head(1)

,question,history_cons,history_lb,no_history_cons,no_history_lb,true
0,Сколько ножек у стола?: Четыре или Сто?,5,1,5,1,1


In [13]:
print(f"Accuracy для сеттинга с показом истории ответов ==> {(df['true'] == df['history_lb']).mean().round(3)} <==")
print(f"Accuracy для сеттинга без показа истории ответов ==> {(df['true'] == df['no_history_lb']).mean().round(3)} <==")

Accuracy для сеттинга с показом истории ответов ==> 0.942 <==
Accuracy для сеттинга без показа истории ответов ==> 0.977 <==


Удивительно, но показ истории немного уменьшил метрики.

### Анализ результатов

Посмотрим на задания, где ответы различаются.

In [14]:
df['history_cor'] = (df['history_lb'] == df['true']) * 1
df['no_history_cor'] = (df['no_history_lb'] == df['true']) * 1

Посмотрим на вопросы, где с историей ошибка, а без — правильно.

In [15]:
len(df[(df['history_cor'] == 0) & (df['no_history_cor'] == 1)])

16

Всего 16 вопросов:

In [16]:
for idx, row in df[(df['history_cor'] == 0) & (df['no_history_cor'] == 1)].iterrows():
    print(f"ВОПРОС: {row['question']}\nОТВЕТ: {row['no_history_lb']}")

ВОПРОС: За сколько времени Земля делает оборот вокруг своей оси?: За 23 часа 56 минут и 4 секунды или За 24 часа 00 минут и 00 секунд?
ОТВЕТ: 1
ВОПРОС: А «КНР»?: Китайская Народная Республика или Корейская Народная Республика?
ОТВЕТ: 1
ВОПРОС: А спички из бумаги?: Нет или Да?
ОТВЕТ: 2
ВОПРОС: А сейчас «пять цифр «восемь»?: Пять цифр «восемь» или 88888?
ОТВЕТ: 1
ВОПРОС: Шахматный конь стоит на клетке d7. На какой клетке у него больше шансов оказаться в следующем ходу, b6 или e8?: b6 или e8?
ОТВЕТ: 1
ВОПРОС: Кто кому приказания отдаёт: старпом боцману или боцман старпому?: Боцман старпому или Старпом боцману?
ОТВЕТ: 2
ВОПРОС: Правда ли, что у светлячков обычно светятся только самки?: Нет или Да?
ОТВЕТ: 1
ВОПРОС: Сколько звонких согласных в слове «колос»?: 3 или 1?
ОТВЕТ: 2
ВОПРОС: Лебедь съел 100 г моллюсков на Сером пруду, 250 г моллюсков на Веронихином пруду, по 50 грамм моллюсков на прудах Приплёсском, Винницыном и Ровном и ещё 200 г на Куньем пруду. Сколько всего прудов облетел лебед

В ошибках нет вопросов на память. Получается, это просто невнимательность или ошибки в знаниях у разметчиков.

Теперь вопросы, где ситуация наоборот.

In [17]:
len(df[(df['history_cor'] == 1) & (df['no_history_cor'] == 0)])

1

In [18]:
for idx, row in df[(df['history_cor'] == 1) & (df['no_history_cor'] == 0)].iterrows():
    print(f"ВОПРОС: {row['question']}\nОТВЕТ: {row['history_lb']}")

ВОПРОС: Если написать подряд буквы «I» и «О» и соединить их горизонтальной чертой, то на какую букву это будет меньше похоже, на Ю или на Б?: Ю или Б?
ОТВЕТ: 2


Всего один вопрос, где с историей ответ правильный, а без истории — нет. Но этот вопрос никак не связан с памятью.

Какие ошибки общие?

In [19]:
len(df[(df['history_cor'] == 0) & (df['no_history_cor'] == 0)])

9

In [20]:
for idx, row in df[(df['history_cor'] == 0) & (df['no_history_cor'] == 0)].iterrows():
    print(f"ВОПРОС: {row['question']}\nОТВЕТ: {row['true']}")

ВОПРОС: 6+2?: 12 или 7?
ОТВЕТ: 2
ВОПРОС: А шесть плюс два?: Семь или Двенадцать?
ОТВЕТ: 1
ВОПРОС: Есть ли у кота шпоры?: Нет или Да?
ОТВЕТ: 1
ВОПРОС: А ежели кит на слона нападёт, кто кого поборет? Ответь на этот вопрос наоборот?: Слон кита или Кит слона?
ОТВЕТ: 1
ВОПРОС: А два кружочка?: Нет или Да?
ОТВЕТ: 1
ВОПРОС: А ковырять в носу?: Да или Нет?
ОТВЕТ: 1
ВОПРОС: Может ли длина окружности быть больше, чем длина полуокружности того же радиуса?: Да или Нет?
ОТВЕТ: 1
ВОПРОС: Сколько ударений в слове автомотоклуб?: Одно или Три?
ОТВЕТ: 2
ВОПРОС: Митя запускал планёр в сторону юга, но встречный ветер погнал его в противоположную сторону. Затем ветер сменился, планёр повлекло направо. Потом он повернул налево, ещё и ещё налево. В каком направлении теперь летит планёр, если он каждая смена направления кратна 90 градусам?: На юг или На запад?
ОТВЕТ: 1


Снова невнимательность, недопонимание инструкции (нужно наиболее близкий к правде искать ответ), а также простые ошибки. Например, кажется, про шпоры у кота разметчики вспомнили "кота в сапогах".